<a href="https://colab.research.google.com/github/Ehizson/HDSC-W2023-Projects/blob/main/Stage_B_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

Energy_data = pd.read_csv('/content/energydata_complete.csv')
Energy_data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [ ]:
X = Energy_data.drop('T6', axis=1)
y = Energy_data['T6']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train[['T2']], y_train)
prediction = model.predict(X_test[['T2']])

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_test, prediction))

0.6427789637197382


In [ ]:
print(round(r2_score(y_test, prediction), 2))

0.64


In [ ]:
copy = Energy_data.copy()

In [ ]:
copy = copy.drop(['date', 'lights'], axis=1)

In [ ]:
# instantiate minmax scalar

from sklearn.preprocessing import MinMaxScaler
min_max = MinMaxScaler()

In [ ]:
scaled_df = min_max.fit_transform(copy)

In [29]:
normalized_df = pd.DataFrame(scaled_df, columns = copy.columns)

In [35]:
x = normalized_df.drop('Appliances', axis=1)
y = normalized_df['Appliances']

In [37]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

In [40]:
from sklearn.metrics import mean_absolute_error

m_2 = LinearRegression()
m_2.fit(x_train, y_train)
prediction = m_2.predict(x_test)
ab_error = round(mean_absolute_error(y_test, prediction), 2)
ab_error

0.05

In [41]:
residual = round(np.sum(np.square(prediction - y_test)), 2)
residual

45.35

In [44]:
from sklearn.metrics import mean_squared_error

root_mse = round(mean_squared_error(y_test, prediction, squared=False), 3)
root_mse

0.088

In [45]:
coefficient = round(r2_score(y_test, prediction), 2)
coefficient

0.15

In [51]:
def feature_weights (model_name, feat, col_name):
    weights = pd.Series(model_name.coef_, feat.columns).sort_values()
    df_weights = pd.DataFrame(weights).reset_index()
    df_weights.columns = ["Features", col_name]
    df_weights[col_name].round(2)
    return df_weights

Regression_model_weights = feature_weights(m_2, x_train, "Regression_model_weights")
Regression_model_weights

,Features,Regression_model_weights
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [54]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)
pred = ridge_reg.predict(x_test)
ridge_error = round(mean_squared_error(y_test, pred, squared=False), 3)
ridge_error

0.088

In [55]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

weights = feature_weights(lasso_reg, x_train, "Weight")
weights

,Features,Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [56]:
prediction = lasso_reg.predict(x_test)
rms_error = round(mean_squared_error(y_test, prediction, squared=False), 3)
rms_error

0.094